https://newsapi.org/

In [1]:
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup
import numpy as np

In [4]:
def google_news_download(palavra):
    url = f"https://newsapi.org/v2/everything?q={palavra}&language=pt&from=2019-10-25&pageSize=100&sortBy=publishedAt&apiKey=c95ae50dc105420ba0263be38e60808c"
    r = requests.get(url)
    if r.status_code == 200:
        return r.json()
    else:
        return None

resultado = None
with ThreadPoolExecutor(max_workers=4) as exc:
    palavras = [
        "bitcoin", "saúde", "política",
        "ciência", "futebol", "tecnologia",
        "google", "apple", "câncer", "preconceito",
        "informação", "cinema", "guerra", "quadrinhos",
        "amor", "economia", "negócio"
    ]
    resultado = list(exc.map(google_news_download, palavras, chunksize=3))

In [5]:
artigos = []
for articles in resultado:
    for article in articles["articles"]:
        titulo = article["title"]
        conteudo = article["content"]
        if titulo and conteudo:
            titulo = titulo.replace("“", '"')
            titulo = titulo.replace("”", '"')
            for k in ["\r\n", "\xa0", "–", ]:
                titulo = titulo.replace(k, "")

            conteudo = conteudo.split("…")[0].strip()
            for k in ["\r\n", "\xa0", "–", ]:
                conteudo = conteudo.replace(k, "")
            conteudo = conteudo.replace("“", '"')
            conteudo = conteudo.replace("”", '"')
            soup = BeautifulSoup(conteudo)
            conteudo = soup.get_text()
            
            artigos.append([titulo, conteudo])
artigos = np.asarray(artigos)

In [6]:
artigos.shape

(1673, 2)

In [7]:
artigos

array([['Novembro está a ser negro para a Bitcoin, com o mercado em queda abrupta',
        'A Bitcoin é a criptomoeda mais famosa do mundo e acaba por influenciar o ritmo do mercado das altcoins. O mês de novembro, especialmente esta última semana, está-se a revelar negro para esta divisa digital.Desde o início do mês, a Bitcoin já desvalorizou ce'],
       ['Bitcoin Cash em foco',
        'O Bitcoin Cash vem perdendo volume há alguns dias e sua cotação enfim acompanhou sua depressão (Imagem: Cryptowatch)O Bitcoin cash (BCH), iniciou uma tendência de alta no final de outubro, contudo o Bitcoin e levou todo o espectro das criptomoedas para o fu'],
       ['Visão de mercado: Bitcoin e Tether',
        'Até o momento, o Bitcoin (BTC) está sendo negociado a US$ 7.290,35 (-4.76%), com um volume diário confiável de US$ 9.53 bilhões. Quanto ao Tether (USDT), ele está sendo negociado a US$ 1.00 (-0.26%), com um volume diário confiável de US$ 10.46 bilhões.'],
       ...,
       ['Manoel Soare

In [8]:
with open("../datasets/gnews.csv", "a+") as fh:
    for artigo in artigos:
        fh.write(f"{artigo[0]};{artigo[1]}\n")

In [9]:
# conteudo = None
# with open("../datasets/gnews.csv", "r") as fh:
#     soup = BeautifulSoup(fh.read())
#     conteudo = soup.get_text()
# with open("../datasets/gnews.csv", "w") as fh:
#     fh.write(conteudo)